In [9]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint
from IPython.display import display
from tqdm import tqdm
import glob
# import mlflow 
from langsmith import Client

%load_ext autoreload
%autoreload 2
%matplotlib inline


sys.path.append(os.path.dirname(os.getcwd()))
from src import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
# use the already-imported src.utils (imported in an earlier cell)
from src.config import load_config

config = load_config()

In [14]:
from src.runner.state import PlannerStateMaker


state_maker = PlannerStateMaker()

state = state_maker.make(user_query="get a apple", context="")

In [16]:
from src.runner.runner import PlannerRunner


runner = PlannerRunner(config=config)

TypeError: PlannerRunner.__init__() got an unexpected keyword argument 'config'